In [106]:
import pandas as pd

filepath = './src/design_parameter_table.csv'
df_dp = pd.read_csv(filepath, header=0, index_col=None)
filepath = './src/product_definition_table.csv'
df_product_definition = pd.read_csv(filepath, header=0, index_col=None)
filepath = './src/product_model_1.csv'
df_product_model = pd.read_csv(filepath, header=0, index_col=None)
df_product_model

,functional_metric_id,Dp1,Dp2,Dp3,Dp4,Dp5,Dp6,Dp7,Dp8,Dp9,...,Dp14,Dp15,Dp16,Dp17,Dp18,Dp19,Dp20,Dp21,Dp22,Dp23
0,Fm1,1,3,1,1,1,1,1,1,1,...,0,0,3,1,1,3,3,3,1,1
1,Fm2,1,1,1,1,1,1,3,3,1,...,0,0,3,1,3,1,3,3,1,1
2,Fm3,0,1,1,1,1,0,1,3,1,...,1,1,0,0,1,0,0,0,1,1
3,Fm4,0,1,1,1,1,0,1,3,1,...,1,1,0,0,1,0,0,0,1,1
4,Fm5,3,1,1,0,0,0,0,1,1,...,0,0,0,0,1,0,0,0,3,1
5,Fm6,3,1,1,3,1,1,0,1,1,...,0,0,0,0,1,0,0,0,3,1
6,Fm7,1,1,0,1,1,0,1,1,0,...,3,3,3,3,1,3,1,1,1,1
7,Fm8,1,1,0,1,1,0,1,1,0,...,3,3,3,3,1,1,1,1,1,1
8,Fm9,1,1,0,1,1,0,1,1,0,...,1,1,3,3,1,3,1,1,1,1
9,Fm10,1,1,0,1,1,0,1,1,0,...,1,1,3,3,1,1,1,1,1,1


In [107]:
for i in range(len(df_product_model.columns)-1):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22


In [108]:
importances = df_product_definition.iloc[:,2:].to_numpy()
sensitivity = df_product_model.iloc[:,1:].to_numpy()

for i in range(len(df_product_definition.columns)-2):
    for j in range(len(df_product_model.columns)-1):
        for k in range(len(df_product_definition)):
    

sensitivity
# importances

IndentationError: expected an indented block after 'for' statement on line 6 (628794127.py, line 9)

In [1]:
import tkinter as tk
import tkinter.ttk as ttk
from tkinter import filedialog
from tkinter import messagebox
import os
import csv
import numpy as np
import random
from IPython import embed
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
from pyvis.network import Network
import datetime
import matplotlib.pyplot as plt
from chardet.universaldetector import UniversalDetector
import re
import math
import itertools
import copy


# DSM関数

## 共通

In [2]:
# class Common():
#     def __init__(self):
#         print("")

def sort_matrix_row(matrix, idx):
    #-----------------------
    # 行方向に降順に並び替える
    # matrix: numpy array型
    # idx: 固定する列インデックス
    #-----------------------
    
    rtn = matrix.copy()

    # Quickソートで昇順に並び替え
    for ir1, mr1 in enumerate(rtn):
        for ir2, mr2 in enumerate(rtn):
            
            if ir2 > ir1 and rtn[ir1, idx] < rtn[ir2, idx]:
                swap = rtn[ir1, :].copy()
                rtn[ir1, :] = rtn[ir2, :].copy()
                rtn[ir2, :] = swap.copy()
    return rtn

def convert_1d_to_2d(l, cols):
    return [l[i:i + cols] for i in range(0, len(l), cols)]


## MDL法

In [3]:
class GeneticAlgorithm():
    def __init__(self, l):
        self.chromosome = np.zeros(l,dtype=int)
        self.evaluation = 0.0
        self.create_chromosome()

    # 染色体をランダムに初期化
    def create_chromosome(self):
        for i, c in enumerate(self.chromosome):
            self.chromosome[i] = random.randrange(2)
        return

    # 突然変異
    def mutation(self, pm):
        for i, c in enumerate(self.chromosome):
            if random.random() < pm:
                self.chromosome[i] = 1 - c
        return                

    # 評価
    def evaluate(self, dsm, alpha, beta):

        # クラスター数の算出，要素のリストアップ
        elms_in_cls = []
        elms_in_cl = []
        n_cluster = 0
        for i, g in enumerate(self.chromosome):
            ic = 1 + math.floor(i / len(dsm))
            ind = i % len(dsm)
            if g == 1:
                elms_in_cl.append(ind)
                n_cluster = ic

            if (i + 1) % len(dsm) == 0:
                elms_in_cls.append(elms_in_cl)
                elms_in_cl = []

        nelms_in_cls = sum(len(v) for v in elms_in_cls)

        # 理想的なDSMの作成
        dsm_ideal = np.zeros((len(dsm), len(dsm)))
        for elms in elms_in_cls:
            for elm1 in elms:
                for elm2 in elms:
                    #elm1==elm2を除けば，対角は0になる
                    dsm_ideal[elm1, elm2] = 1

        # s1とs2の算出
        s1 = 0
        s2 = 0
        for i, d in enumerate(dsm_ideal):
            for j, e in enumerate(d):
                if i != j:
                    if dsm[i, j] < dsm_ideal[i, j]:
                        s1 += dsm_ideal[i,j] - dsm[i, j]
                    elif dsm[i, j] > dsm_ideal[i, j]:
                        s2 += dsm[i, j] - dsm_ideal[i, j]

        self.evaluation = \
            (1.0 - alpha - beta) * (n_cluster * math.log2(len(dsm)) + math.log2(len(dsm)) * nelms_in_cls) \
            + alpha * s1 * (2.0 * math.log2(len(dsm)) + 1.0) \
            + beta * s2 * (2.0 * math.log2(len(dsm)) + 1.0)

        return


In [4]:
class MinimumDescriptionLength():

    def __init__(self, elms, values, pc, pm, lmbd, mu, alpha, beta, c_max, nrep):
        self.label = elms
        self.dsm = values

        self.pc = int(pc)
        self.pm = float(pm)
        self.lmbd = int(lmbd)
        self.mu = int(mu)
        self.alpha = float(alpha)
        self.beta = float(beta)
        self.c_max = int(c_max)
        self.nrep = int(nrep)
        
    def crossover(self, gas):
        ga_child1 = GeneticAlgorithm(len(self.label)*self.c_max)
        ga_child2 = GeneticAlgorithm(len(self.label)*self.c_max)
        ga_child1.chromosome = gas[0].chromosome.copy()
        ga_child2.chromosome = gas[1].chromosome.copy()

        for i, c1 in enumerate(ga_child1.chromosome):
            if np.random.rand() < 0.5:
                ga_child1.chromosome[i] = ga_child2.chromosome[i]
                ga_child2.chromosome[i] = c1

        return [ga_child1, ga_child2]     

    def select_elite(self, gas):

        # 並び替え用のデータフレームを作成
        l1 = []
        l2 = []
        for i, ga in enumerate(gas):
            l1.append(i)
            l2.append(ga.evaluation)
        df = pd.DataFrame(zip(l1,l2), columns=['key', 'value'])            
        df_s = df.sort_values('value') # 評価値の低い順にソート

        # 評価の良い順に格納
        selected_generations = []
        cnt = 0
        for index, row in df_s.iterrows():
            selected_generations.append(gas[int(row['key'])])
            cnt += 1
            if cnt >= self.lmbd:
                break

        return selected_generations

    def main(self):
        # 初期個体
        current_generations = [GeneticAlgorithm(len(self.label)*self.c_max) for i in range(self.lmbd)]

        # 初期個体の適応度計算
        for ga in current_generations:
            ga.evaluate(self.dsm, self.alpha, self.beta)
        
        # 評価履歴格納用
        evaluation_min = []
        evaluation_max = []
        evaluation_ave = []

        # 反復
        for irep in range(self.nrep):

            # 一様交叉
            if random.random() < self.pc:
                
                # 交叉するすべての組み合わせを設定
                ga_pairs = []
                for ga_pair in itertools.combinations(current_generations, 2):
                    ga_pairs.append(ga_pair)

                # すべての組み合わせからmu/2個選定
                ga_pairs_selected = random.sample(ga_pairs, int(self.mu/2))

                # 交叉
                progeny_generations = []
                for ga_pair in ga_pairs_selected: # pairはタプル
                    # print(ga_pair[0].chromosome)
                    progeny_generations.extend(self.crossover(ga_pair))

            for ga in progeny_generations:
                # 突然変異
                ga.mutation(self.pm)
    
                # 適応度計算
                ga.evaluate(self.dsm, self.alpha, self.beta)

            # 現役世代と次世代を格納した変数の設定
            tmp_generations = current_generations + progeny_generations

            # エリート選択（淘汰）
            current_generations = self.select_elite(tmp_generations)
            
            # 評価履歴格納
            cnt = 0
            tmp_min = 10000000
            tmp_max = 0
            tmp_ave = 0
            for ga in current_generations:
                if ga.evaluation <= tmp_min:
                    tmp_min = ga.evaluation
                    ga_best = ga
                if ga.evaluation >= tmp_max:
                    tmp_max = ga.evaluation
                tmp_ave = tmp_ave + ga.evaluation
                cnt += 1
            tmp_ave = tmp_ave / cnt
            evaluation_min.append(tmp_min)
            evaluation_max.append(tmp_max)
            evaluation_ave.append(tmp_ave)

            # 表示
            if irep % 10 == 0:
                print("----- 第" + str(irep) + "世代の結果 -----")
                print("  Min:" + str(tmp_min))
                print("  Max:" + str(tmp_max))
                print("  Avg:" + str(tmp_ave))


        # 出力
        os.makedirs('./out', exist_ok=True)
        now = datetime.datetime.now().strftime(r'%Y%m%d_%H%M%S') # 現在時刻を年月曜日で表示
        
        with open('./out/mdl_param_' + now + '.csv', 'wt', encoding='utf-8') as f:
            # ライター（書き込み者）を作成
            writer = csv.writer(f)
            # ライターでデータ（リスト）をファイルに出力
            writer.writerow(['pc', self.pc])
            writer.writerow(['pm', self.pm])
            writer.writerow(['lmbd', self.lmbd])
            writer.writerow(['mu', self.mu])
            writer.writerow(['alpha', self.alpha])
            writer.writerow(['beta', self.beta])
            writer.writerow(['c_max', self.c_max])
            writer.writerow(['nrep', self.nrep])
            
        df_cost_log = pd.DataFrame(zip(evaluation_min,evaluation_max,evaluation_ave), columns=['min','max','average'])
        df_cost_log.to_csv('./out/mdl_cost_' + now + '.csv', sep=',', header=True, index=True, encoding='utf-8')
        plt.figure()
        df_cost_log.plot()
        plt.savefig('./out/mdl_cost_' + now + '.png')
        plt.close('all')

        # 再配列
        # !!!! 未実装 !!!!!

        # 出力
        with open('./out/mdl_chromosomes_' + now + '.csv', 'wt', encoding='utf-8') as f:
            # ライター（書き込み者）を作成
            writer = csv.writer(f)
            # ライターでデータ（リスト）をファイルに出力
            for ga in current_generations:
                writer.writerow(ga.chromosome)

        best_chromosome = convert_1d_to_2d(ga_best.chromosome, len(self.label))
        df_cluster_matrix = pd.DataFrame(best_chromosome, columns=self.label)
        df_cluster_matrix.to_csv('./out/mdl_best_cluster_matrix_' + now + '.csv', sep=',', header=True, index=True, encoding='utf-8')

        return
    

In [5]:
values = np.eye(5, 5)
values[0,1] = 1
values[1,0] = 1
values[0,2] = 1
values[2,0] = 1
values[1,2] = 1
values[2,1] = 1
values[3,4] = 1
values[4,3] = 1
print(values)
elms = ['要素A','要素B','要素C','要素D','要素E']

df_test = pd.DataFrame(values, columns=elms, index=elms)
# df_test


mdl = MinimumDescriptionLength(elms, values, 1, 0.03, 100, 100, 0.33, 0.33, 2, 300)
mdl.main()


[[1. 1. 1. 0. 0.]
 [1. 1. 1. 0. 0.]
 [1. 1. 1. 0. 0.]
 [0. 0. 0. 1. 1.]
 [0. 0. 0. 1. 1.]]
----- 第0世代の結果 -----
  Min:5.526188865831921
  Max:24.150914292088512
  Avg:20.225609064212744
----- 第10世代の結果 -----
  Min:5.526188865831921
  Max:5.526188865831921
  Avg:5.526188865831915
----- 第20世代の結果 -----
  Min:5.526188865831921
  Max:5.526188865831921
  Avg:5.526188865831915
----- 第30世代の結果 -----
  Min:5.526188865831921
  Max:5.526188865831921
  Avg:5.526188865831915
----- 第40世代の結果 -----
  Min:5.526188865831921
  Max:5.526188865831921
  Avg:5.526188865831915
----- 第50世代の結果 -----
  Min:5.526188865831921
  Max:5.526188865831921
  Avg:5.526188865831915
----- 第60世代の結果 -----
  Min:5.526188865831921
  Max:5.526188865831921
  Avg:5.526188865831915
----- 第70世代の結果 -----
  Min:5.526188865831921
  Max:5.526188865831921
  Avg:5.526188865831915
----- 第80世代の結果 -----
  Min:5.526188865831921
  Max:5.526188865831921
  Avg:5.526188865831915
----- 第90世代の結果 -----
  Min:5.526188865831921
  Max:5.526188865831921
  

In [6]:
class Test(): 

    def __init__(self):
        self.a = 10
        self.b = [1,2]
    def change(self):
        self.a = 11
    def copy(self):
        return self
    
t1 = Test()
t2 = t1
t3 = copy.copy(t1)
t4 = copy.deepcopy(t1)
t1.a = 11
t1.b.append(3)
print(t1.b)
print(t2.b)
print(t3.b)
print(t4.b)
# l = [Test(), Test()]

# print(l[0].a)

# for l1 in l:
#     l1.a = 11
#     # l1.change()

# print(l[0].a)


[1, 2, 3]
[1, 2, 3]
[1, 2, 3]
[1, 2]


## TCC法

In [7]:
class TotalCoordinationCost(): 

    def __init__(self, elms, values, npow_cc=1, npow_bid=0, npow_dep=1, nrand_accept=30, ntimes=5):
        self.label = elms
        self.dsm = values
        
        self.npow_cc = float(npow_cc)
        self.npow_bid = float(npow_bid)
        self.npow_dep = float(npow_dep)
        self.nrand_accept = int(nrand_accept)
        self.ntimes = int(ntimes)
        
        self.cluster_matrix = np.eye(len(self.label))
        self.cost = self.total_coordination_cost()

        self.cost_log = np.zeros(self.ntimes * len(self.label))

        self.label_ordered = self.label.copy()
        self.dsm_ordered = self.dsm.copy()
        self.cluster_matrix_ordered = np.eye(len(self.label))
        # embed()


    def find_elements_in_cluster(self, matrix, no_key, idx, is_row):
        # ====================================================
        # 指定の値(key)に等しい要素の行番号/列番号を配列で返す関数
        #  no_key    : 含まない値
        #  idx       : 調査対象の行もしくは列
        #  is_row    : idxは行か列か（True:行,false:列)
        # ====================================================
        elements_ids = []

        if is_row == False:
            for ir in range(0,len(self.label)):
                if matrix[ir, idx] != no_key:
                    elements_ids.append(ir)
        else:
            for ic in range(0,len(self.label)):
                if matrix[idx, ic] != no_key:
                    elements_ids.append(ic)

        return elements_ids

    def cluster_bid(self, element_id):
        best_cluster_id = -1
        best_bid = -1

        for icl in range(0, len(self.label)):
            
            # i番目のクラスターに属する要素のリストを作成
            elms_in_cluster = self.find_elements_in_cluster(self.cluster_matrix ,0, icl, True)

            if len(elms_in_cluster) > 0:
                bid = 0
                for i, elm in enumerate(elms_in_cluster):
                    #!!!! 要検討 2校目を追加したが正しいか？ !!!! 
                    bid += self.dsm[element_id, elm] + self.dsm[elm, element_id] #2項目追加

                bid = (bid ** self.npow_dep) / (len(elms_in_cluster) ** self.npow_bid)

                if bid > best_bid:
                    best_bid = bid
                    best_cluster_id = icl
                    
        return best_cluster_id

    def total_coordination_cost(self):
        intra_cost = 0.0
        extra_cost = 0.0
        
        for i in range(0, len(self.label)):
            # ここはVBAから変更した．重複がないようにiとjで．
            for j in range(i + 1, len(self.label)):
            # for j in range(0, len(self.label)):
                cluster_ofi = self.find_elements_in_cluster(self.cluster_matrix, 0, i, False)
                cluster_ofj = self.find_elements_in_cluster(self.cluster_matrix, 0, j, False)

                if len(cluster_ofi) > 0 and len(cluster_ofj) > 0:
                    cost = self.dsm[i, j] + self.dsm[j,i]

                    if cluster_ofi == cluster_ofj:
                        intra_cost += cost * (len(cluster_ofi) ** self.npow_cc)
                    else:
                        extra_cost += cost * (len(self.label) ** self.npow_cc)
                else:
                    print("まだ未対応")
                    embed()

        return intra_cost + extra_cost

    def reorder_cluster_DSM(self):

        # 1. 並び替え用のクラスターマトリクスを作成
        # (1) クラスターマトリクスに，クラスターの属する個数を併記(@ゼロ列目)した配列
        cluster_matrix_tmp = np.zeros((len(self.label), len(self.label) + 1))
        for i, ci in enumerate(self.cluster_matrix):
            for j, cij in enumerate(ci):
                cluster_matrix_tmp[i, 0] += cij
                cluster_matrix_tmp[i, j + 1] = cij

        # (2) 並び替え用の配列の０列目を基準に並び替え
        cluster_matrix_tmp = sort_matrix_row(cluster_matrix_tmp, 0)

        # (3) 並び替え後のクラスターマトリクス
        self.cluster_matrix_ordered[:,:] = cluster_matrix_tmp[:,1:].copy()

        elms_order = []
        for icl in range(0, len(self.label)):
            elms_order.extend(self.find_elements_in_cluster(self.cluster_matrix_ordered, 0, icl, True)) #icl行目の0でない値の列番号を取ってくる．

        # 2. 並び替え後のラベル
        self.label_ordered = [self.label[i] for i in elms_order]

        # 3. 並び替え後のDSM
        rep = np.zeros((len(self.label), len(self.label)))
        eigen_mat = np.eye(len(self.label))
        for i in range(len(self.label)):
            for j in range(len(self.label)):
                rep[i, j] = eigen_mat[elms_order[i],j]
        self.dsm_ordered = np.dot(np.dot(rep, self.dsm), rep.T)

        return

    def main(self):

        for i, cost in enumerate(self.cost_log):
            pre_cluster_matrix = self.cluster_matrix
            
            # ランダムに選んだ要素に対して，ベストなクラスターidを算出する
            element_id = random.randrange(0, len(self.label))
            best_cluster_id = self.cluster_bid(element_id)
            # クラスターマトリクスの更新
            self.cluster_matrix[:, element_id] = 0
            self.cluster_matrix[best_cluster_id, element_id] = 1

            cost_new = self.total_coordination_cost()

            # コスト判定
            if self.cost <= max(cost, self.nrand_accept):
                # コストが低くなった場合，低いコストを現状のコストにする
                self.cost = cost_new
            else:
                # コストが高くなった場合，元のクラスタに戻す
                self.cluster_matrix = pre_cluster_matrix

            self.cost_log[i] = cost_new

        # 出力
        os.makedirs('./out', exist_ok=True)
        now = datetime.datetime.now().strftime(r'%Y%m%d_%H%M%S') # 現在時刻を年月曜日で表示
        
        with open('./out/tcc_param_' + now + '.csv', 'wt', encoding='utf-8') as f:
            # ライター（書き込み者）を作成
            writer = csv.writer(f)
            # ライターでデータ（リスト）をファイルに出力
            writer.writerow(['npow_cc', self.npow_cc])
            writer.writerow(['npow_bid', self.npow_bid])
            writer.writerow(['npow_dep', self.npow_dep])
            writer.writerow(['nrand_accept', self.nrand_accept])
            writer.writerow(['ntimes', self.ntimes])

        df_cost_log = pd.DataFrame(self.cost_log, columns=['cost'])
        df_cost_log.to_csv('./out/tcc_cost_' + now + '.csv', sep=',', header=False, index=True, encoding='utf-8')
        plt.figure()
        df_cost_log.plot()
        plt.savefig('./out/tcc_cost_' + now + '.png')
        plt.close('all')

        # 再配列
        self.reorder_cluster_DSM()

        # 出力（クラスターマトリクスは，再配列されたもののほうが見やすいので）
        df_cluster_matrix = pd.DataFrame(self.cluster_matrix_ordered, columns=self.label)
        df_cluster_matrix.to_csv('./out/tcc_cluster_matrix_' + now + '.csv', sep=',', header=True, index=True, encoding='utf-8')


        # embed()
        
        return

In [8]:
values = np.eye(5, 5)
values[0,1] = 1
values[0,3] = 1
values[1,2] = 1
values[1,4] = 1
values[2,1] = 1
values[2,4] = 1
print(values)
elms = ['要素A','要素B','要素C','要素D','要素E']

df_test = pd.DataFrame(values, columns=elms, index=elms)
# df_test



# tcc = TotalCoordinationCost(elms, values)
# print(tcc.cluster_matrix)

[[1. 1. 0. 1. 0.]
 [0. 1. 1. 0. 1.]
 [0. 1. 1. 0. 1.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]


# メニュー部

## DSM表示部

In [9]:
class AppDSM(ttk.Frame): #ttk.Frameを継承したクラスになっている

    def __init__(self,master):
        super().__init__(master,borderwidth=5)

        # 変数
        self.square_size = 20 # マスのサイズ
        self.canvas_size = self.square_size * 5
        self.canvas_size_elm_width = 200 # 要素部（縦）キャンバスの幅
        self.canvas_size_elm_height = 200 # 要素部（横）キャンバスの高さ
        self.DSM_values = np.zeros((5, 5),dtype = 'int64')
        self.DSM_elms = ['要素A','要素B','要素C','要素D','要素E']
        self.idx_elm_selected_vertical = -1
        self.idx_elm_selected_horizontal = -1

        self.create_widgets_DSM()
        self.create_widgets_operations()
        self.set_events_DSM()
        self.set_events_operations()
        self.init_DSM()

    def click_DSM(self, event):
        # クリックされた位置がどのマスであるかを計算
        self.idx_elm_selected_horizontal = event.x // self.square_size
        self.idx_elm_selected_vertical = event.y // self.square_size
        self.highlight_selected()


    def highlight_selected(self):

        # 対応する要素をハイライト
        for i in range(self.num_square):
            tag_name = 'elm_vertical_' + str(i)
            if i == self.idx_elm_selected_vertical:
                self.canvas_elm_vertical.itemconfig(
                    tag_name,
                    fill="red"
                )
            else:
                self.canvas_elm_vertical.itemconfig(
                    tag_name,
                    fill="white"
                )

        for i in range(self.num_square):
            tag_name = 'elm_horizontal_' + str(i)
            if i == self.idx_elm_selected_horizontal:
                self.canvas_elm_horizontal.itemconfig(
                    tag_name,
                    fill="red"
                )
            else:
                self.canvas_elm_horizontal.itemconfig(
                    tag_name,
                    fill="white"
                )

        for ix in range(self.num_square):
            for iy in range(self.num_square):
                tag_name = 'DSM_' + str(ix) + '_' + str(iy)
                if ix == iy:
                    self.canvasDSM.itemconfig(
                        tag_name,
                        fill="black"
                    )
                elif self.idx_elm_selected_vertical == iy and self.idx_elm_selected_horizontal >= ix:
                    self.canvasDSM.itemconfig(
                        tag_name,
                        fill="red"
                    )
                elif self.idx_elm_selected_horizontal == ix and self.idx_elm_selected_vertical >= iy:
                    self.canvasDSM.itemconfig(
                        tag_name,
                        fill="red"
                    )

                else:
                    if self.DSM_values[ix][iy] == 0:
                        self.canvasDSM.itemconfig(
                            tag_name,
                            fill="#555"
                        )
                    else:
                        self.canvasDSM.itemconfig(
                            tag_name,
                            fill="white"
                        )


    def click_operation(self, event):

        # ボタンの種類の判別
        if str(event.widget["text"]) == '↑':
            updown = -1
        else:
            updown = 1


        # 選択要素を特定
        if self.idx_elm_selected_vertical == -1:
            messagebox.showerror('並び替えエラー', '先に要素を選択してください')
            return
        elif updown == -1 and self.idx_elm_selected_vertical == 0:
            return
        elif updown == 1 and self.idx_elm_selected_vertical == len(self.DSM_elms) - 1:
            return

        # DSM_values, DSM_elmsを変換
        (self.DSM_values, self.DSM_elms) = sort_DSM(self.DSM_values, self.DSM_elms, self.idx_elm_selected_vertical, updown)

        # 再描写
        # → 消してやり直すか，特定の要素だけ変更するか．．．
        #  枠は消さなくてOKだが，結局ますの色が変わるので，，，
        self.canvasDSM.delete('all')
        self.canvas_elm_vertical.delete('all')
        self.canvas_elm_horizontal.delete('all')
        self.canvas_elm_top.delete('all')

        self.create_widgets_DSM()
        self.set_events_DSM()
        self.init_DSM()

        if self.idx_elm_selected_horizontal == self.idx_elm_selected_vertical:
            # 同じ要素を選択していた場合
            self.idx_elm_selected_horizontal = self.idx_elm_selected_horizontal + updown

        elif self.idx_elm_selected_horizontal == self.idx_elm_selected_vertical + updown:
            # 入れ替わる要素を選択していた場合
            self.idx_elm_selected_horizontal = self.idx_elm_selected_horizontal - updown

        self.idx_elm_selected_vertical = self.idx_elm_selected_vertical + updown

        self.highlight_selected()

    def import_DSM(self, values, elms):
        self.DSM_values = values
        self.DSM_elms = elms
        
        self.canvasDSM.delete('all')
        self.canvas_elm_vertical.delete('all')
        self.canvas_elm_horizontal.delete('all')
        self.canvas_elm_top.delete('all')

        self.create_widgets_DSM()
        self.set_events_DSM()
        self.init_DSM()

    def create_widgets_operations(self):
        self.grid(row=0, column=0, rowspan=2)
        frame_operation = ttk.Frame(self)
        frame_operation.pack()

        self.label_operation = ttk.Label(frame_operation, text="選択要素を移動")
        self.label_operation.pack(side=tk.TOP, expand = True)

        self.button_up = ttk.Button(frame_operation, text="↑")
        self.button_up.pack(side=tk.TOP, expand = True)

        self.button_down = ttk.Button(frame_operation,text="↓")
        self.button_down.pack(side=tk.TOP, expand = True)


    def create_widgets_DSM(self):
        
        self.num_square = len(self.DSM_elms) # 横方向・縦方向のマスの数
        self.canvas_size = self.num_square*self.square_size # DSMキャンバスのサイズ

        # DSM値部キャンバスの作成
        self.canvasDSM = tk.Canvas(
            self.master,
            bg='white',
            width=self.canvas_size+1, # +1は枠線描画のため
            height=self.canvas_size+1, # +1は枠線描画のため
            highlightthickness=0
        )
        self.canvasDSM.grid(row=1, column=2, sticky=tk.N+tk.W)

        # DSM要素部(縦) キャンバスの作成
        self.canvas_elm_vertical = tk.Canvas(
            self.master,
            bg='white',
            width=self.canvas_size_elm_width+1, # +1は枠線描画のため
            height=self.canvas_size+1, # +1は枠線描画のため
            highlightthickness=0
        )
        self.canvas_elm_vertical.grid(row=1, column=1, sticky=tk.N+tk.W)

        # DSM要素部(横) キャンバスの作成
        self.canvas_elm_horizontal = tk.Canvas(
            self.master,
            bg='white',
            width=self.canvas_size+1, # +1は枠線描画のため
            height=self.canvas_size_elm_height+1, # +1は枠線描画のため
            highlightthickness=0
        )
        self.canvas_elm_horizontal.grid(row=0, column=2,sticky=tk.S+tk.W)

        # DSMの左上 キャンバスの作成
        self.canvas_elm_top = tk.Canvas(
            self.master,
            bg='white',
            width=self.canvas_size_elm_width+1, # +1は枠線描画のため
            height=self.canvas_size_elm_height+1, # +1は枠線描画のため
            highlightthickness=0
        )
        self.canvas_elm_top.grid(row=0, column=1)

    def set_events_DSM(self):
        # DSM値部キャンバス上のマウスクリックを受け付ける
        self.canvasDSM.bind('<1>', self.click_DSM)

    def set_events_operations(self):
        self.button_up.bind('<1>', self.click_operation)
        self.button_down.bind('<1>', self.click_operation)


    def init_DSM(self):

        # マスを描画(DSM値部)
        for iy in range(self.num_square):
            for ix in range(self.num_square):
                # 長方形の開始・終了座標を計算
                xs = ix * self.square_size
                ys = iy * self.square_size
                xe = (ix + 1) * self.square_size
                ye = (iy + 1) * self.square_size
                
                # 長方形を描画
                tag_name = 'DSM_' + str(ix) + '_' + str(iy)
                if ix == iy:
                    color = 'black'
                elif self.DSM_values[ix][iy] == 0:
                    color = '#555'
                else:
                    color = 'white'

                self.canvasDSM.create_rectangle(
                    xs, ys,
                    xe, ye,
                    tag = tag_name,
                    fill = color, outline ='black'
                )

        # マスを描画(DSM要素部)
        for i in range(self.num_square):
            # 長方形の開始・終了座標を計算
            pos_beg = i * self.square_size
            pos_end = (i + 1) * self.square_size
            
            # 長方形を描画
            tag_name_v = 'elm_vertical_' + str(i)
            tag_name_h = 'elm_horizontal_' + str(i)

            self.canvas_elm_vertical.create_rectangle(
                0, pos_beg,
                self.canvas_size_elm_width, pos_end,
                tag = tag_name_v,
                fill = "white", outline ='black'
            )
            self.canvas_elm_horizontal.create_rectangle(
                pos_beg, 0,
                pos_end, self.canvas_size_elm_height,
                tag = tag_name_h,
                fill = "white", outline ='black'
            )
            self.canvas_elm_top.create_rectangle(
                0, 0,
                self.canvas_size_elm_width, self.canvas_size_elm_height,
                tag = "top",
                fill = "white", outline ='black'
            )

        # DSM値の初期化
        for ix in range(self.num_square):
            for iy in range(self.num_square):
                tag_name = 'DSM_value_' + str(ix) + '_' + str(iy)

                # 長方形の開始・終了座標を計算
                xc = (ix + 0.5) * self.square_size
                yc = (iy + 0.5) * self.square_size
                self.canvasDSM.create_text(xc, yc, text=str(self.DSM_values[ix][iy]),
                    width=self.square_size, fill='black',tag=tag_name)
        
        # 要素名
        for i in range(self.num_square):
            # 長方形の開始・終了座標を計算
            pos_center = (i + 0.5) * self.square_size
            self.canvas_elm_vertical.create_text(self.canvas_size_elm_width/2, pos_center, text=self.DSM_elms[i],
                width=self.canvas_size_elm_width, fill="black",tag="tag")
            self.canvas_elm_horizontal.create_text(pos_center, self.canvas_size_elm_height/2, text=self.DSM_elms[i],
                width=self.canvas_size_elm_height, fill="black",tag="tag",angle=90)
        
        self.canvas_elm_top.create_text(self.canvas_size_elm_width/2, self.canvas_size_elm_height/2, text="",
            width=self.canvas_size_elm_width, fill="black",tag="tag")


## メニュー駆動部

In [10]:
class AppMenu(ttk.Frame):
    def __init__(self, master = None):
        super().__init__(master)
        self.master.title("Design Structure Matrix")
        self.master.geometry("800x500")

        # メニューバーの作成
        menubar = tk.Menu(self)

        # ファイル
        menu_file = tk.Menu(menubar, tearoff = False)
        menu_file.add_command(label = "インポート",  command = self.menu_file_open_click,  accelerator="Ctrl+O")
        menu_file.add_command(label = "エクスポート", command = self.menu_file_saveas_click, accelerator="Ctrl+S")
        menu_file.add_separator() # 仕切り線
        menu_file.add_command(label = "終了", command = self.master.destroy)
        # ショートカットキーの関連付け
        menu_file.bind_all("<Control-o>", self.menu_file_open_click)
        menu_file.bind_all("<Control-s>", self.menu_file_saveas_click)

        # クラスタリング
        menu_clustering = tk.Menu(menubar, tearoff = False)
        menu_clustering.add_command(label = "Coordination Cost法",  command = self.tcc_click)
        menu_clustering.add_command(label = "Minimum Description Length法",  command = self.mdl_click)
        menu_clustering.add_command(label = "評価のみ（Coordination Cost法）",  command = self.evaluate_tcc_click)
        menu_clustering.add_command(label = "評価のみ（Minimum Description Length法）",  command = self.evaluate_mdl_click)

        # 可視化
        menu_visualize = tk.Menu(menubar, tearoff = False)
        menu_visualize.add_command(label = "ネットワーク図(pyviz)",  command = self.pyviz_click)

        # 表示(Checkbutton)
        menu_disp = tk.Menu(menubar, tearoff = False)
        self.disp1_value = tk.BooleanVar()
        self.disp2_value = tk.BooleanVar()
        self.disp3_value = tk.BooleanVar()
        menu_disp.add_checkbutton(label = "表示１", command = self.menu_disp1_click, variable = self.disp1_value)
        menu_disp.add_checkbutton(label = "表示２", command = self.menu_disp2_click, variable = self.disp2_value)
        menu_disp.add_checkbutton(label = "表示３", command = self.menu_disp3_click, variable = self.disp3_value)

        # 選択(Radiobutton)
        self.radio_val = tk.IntVar() # ラジオボタンの値
        menu_select = tk.Menu(menubar, tearoff = False)
        menu_select.add_radiobutton(label = "選択１", command = self.menu_select_click, variable = self.radio_val, value = 1)
        menu_select.add_radiobutton(label = "選択２", command = self.menu_select_click, variable = self.radio_val, value = 2)
        menu_select.add_radiobutton(label = "選択３", command = self.menu_select_click, variable = self.radio_val, value = 3)

        # メニューバーに各メニューを追加
        menubar.add_cascade(label="ファイル", menu = menu_file)
        menubar.add_cascade(label="クラスタリング", menu = menu_clustering)
        menubar.add_cascade(label="グラフ化", menu = menu_visualize)
        menubar.add_cascade(label="表示", menu = menu_disp)
        menubar.add_cascade(label="選択", menu = menu_select)

        # 親ウィンドウのメニューに、作成したメニューバーを設定
        self.master.config(menu = menubar)

        # 他のフレームの作成
        self.app_DSM = AppDSM(self.master)

    def menu_file_open_click(self, event=None):
        filepath = filedialog.askopenfilename(
            title = "ファイルを開く",
            filetypes=[("csv files","*.csv")],
            initialdir = os.getcwd()
            )
        
        # utf-8かどうかチェック
        with open(filepath, 'rb') as f:  
            detector = UniversalDetector()
            for line in f:
                detector.feed(line)
                if detector.done:
                    break   
            detector.close()
            result = detector.result
        if result['encoding'] == 'SHIFT_JIS':
            encoding = 'CP932'
        else:
            encoding = 'utf-8'

        # １行目に要素ラベル，２行目以降に値となっているかチェックする

        # 値読み込み
        values = np.loadtxt(filepath, delimiter=',', dtype='int64', skiprows=1).T
        print(values)
        # 対角要素は1にする
        for i, row in enumerate(values):
            for j, c in enumerate(row):
                if i == j:
                    values[i, j] = 1
        print(values)

        # 要素ラベル読み込み
        with open(filepath, encoding=encoding, newline='') as f:
        # with open(filepath, encoding='utf8', newline='') as f:
            row1 = next(csv.reader(f, delimiter=','))
        elms = row1

        # 不正なファイルでないかチェックする
        if values.shape[0] != values.shape[1]:
            messagebox.showerror('インポートエラー', 'DSMが正方行列ではありません')
            return
        elif values.shape[0] != len(elms):
            messagebox.showerror('インポートエラー', '要素ラベルの数とDSMの行列数が異なります')
            return

        self.app_DSM.import_DSM(values, elms)
        messagebox.showinfo('', 'DSMがインポートされました')

    def menu_file_saveas_click(self, event=None):
        os.makedirs('./out', exist_ok=True)
        now = datetime.datetime.now().strftime(r'%Y%m%d_%H%M%S') # 現在時刻を年月曜日で表示

        df_dsm = pd.DataFrame(self.app_DSM.DSM_values, index=self.app_DSM.DSM_elms, columns=self.app_DSM.DSM_elms)
        df_dsm.to_csv('./out/dsm_' + now + '.csv', sep=',', header=True, index=False, encoding='utf-8')
        messagebox.showinfo('', 'DSMがエクスポートされました')


    def invalidText(self):
        messagebox.showwarning('警告','数値を入力してください')

    def onValidate(self, S):
        # 入力された文字が半角数字の場合
        # reについて : https://note.nkmk.me/python-re-match-search-findall-etc/
        if re.match(re.compile('[0-9]+'), S):
            return True
        elif S == r'.':
            return True
        else:
            # 入力不正のブザーを鳴らす。
            self.bell()
            return False

    def tcc_click(self, event=None):
        self.app_tcc_param = tk.Tk()       
        self.app_tcc_param.title("クラスタリング(Total Coordination)パラメータ")
        self.app_tcc_param.geometry("500x300")
        self.app_tcc_param.attributes("-topmost", True)

        # メインフレームの作成と設置
        vcmd = self.register(self.onValidate)
        
        frame_app = ttk.Frame(self.app_tcc_param)
        frame_app.pack()
        label_npow_cc = tk.Label(frame_app, text='npow_cc')
        label_npow_cc.grid(row=0, column=0)
        self.entry_npow_cc = tk.Entry(frame_app)
        vcmd = (self.entry_npow_cc.register(self.onValidate), '%S')
        self.entry_npow_cc.configure(validate='key', vcmd=vcmd)
        self.entry_npow_cc.grid(row=0, column=1)
        self.entry_npow_cc.insert(0, 1)

        label_npow_bid = tk.Label(frame_app, text='npow_bid')
        label_npow_bid.grid(row=1, column=0)
        self.entry_npow_bid = tk.Entry(frame_app)
        vcmd = (self.entry_npow_bid.register(self.onValidate), '%S')
        self.entry_npow_bid.configure(validate='key', vcmd=vcmd)
        self.entry_npow_bid.grid(row=1, column=1)
        self.entry_npow_bid.insert(0, "0")

        label_npow_dep = tk.Label(frame_app, text='npow_dep')
        label_npow_dep.grid(row=2, column=0)
        self.entry_npow_dep = tk.Entry(frame_app)
        vcmd = (self.entry_npow_dep.register(self.onValidate), '%S')
        self.entry_npow_dep.configure(validate='key', vcmd=vcmd)
        self.entry_npow_dep.grid(row=2, column=1)
        self.entry_npow_dep.insert(0, "1")

        label_nrand_accept = tk.Label(frame_app, text='nrand_accept')
        label_nrand_accept.grid(row=3, column=0)
        self.entry_nrand_accept = tk.Entry(frame_app)
        vcmd = (self.entry_nrand_accept.register(self.onValidate), '%S')
        self.entry_nrand_accept.configure(validate='key', vcmd=vcmd)
        self.entry_nrand_accept.grid(row=3, column=1)
        self.entry_nrand_accept.insert(0, "30")

        label_ntimes = tk.Label(frame_app, text='ntimes')
        label_ntimes.grid(row=4, column=0)
        self.entry_ntimes = tk.Entry(frame_app)
        vcmd = (self.entry_ntimes.register(self.onValidate), '%S')
        self.entry_ntimes.configure(validate='key', vcmd=vcmd)
        self.entry_ntimes.grid(row=4, column=1)
        self.entry_ntimes.insert(0, "5")

        button_execute = ttk.Button(frame_app, text="実行", command=self.tcc_exec)
        button_execute.grid(row=5, column=0)
        
        # self.app_tcc_param.protocol("WM_DELETE_WINDOW") #ウィンドウ削除
        self.app_tcc_param.mainloop()

    def tcc_exec(self):
        npow_cc = self.entry_npow_cc.get()
        npow_bid = self.entry_npow_bid.get()
        npow_dep = self.entry_npow_dep.get()
        nrand_accept = self.entry_nrand_accept.get()
        ntimes = self.entry_ntimes.get()

        tcc = TotalCoordinationCost(
                self.app_DSM.DSM_elms, 
                self.app_DSM.DSM_values,
                npow_cc,
                npow_bid,
                npow_dep,
                nrand_accept,
                ntimes)
        tcc.main()

        # 再配列されたDSMを表示する
        self.app_DSM.import_DSM(tcc.dsm_ordered.astype('int64'), tcc.label_ordered)

        self.app_tcc_param.destroy()
        messagebox.showinfo('', 'クラスタリング(TCC)が完了し，実行ログを出力しました')

    def mdl_click(self, event=None):
        self.app_mdl_param = tk.Tk()       
        self.app_mdl_param.title("クラスタリング(Minimum Description Length)パラメータ")
        self.app_mdl_param.geometry("500x300")
        self.app_mdl_param.attributes("-topmost", True)

        # メインフレームの作成と設置
        vcmd = self.register(self.onValidate)
        
        frame_app = ttk.Frame(self.app_mdl_param)
        frame_app.pack()
        label_pc = tk.Label(frame_app, text='pc')
        label_pc.grid(row=0, column=0)
        self.entry_pc = tk.Entry(frame_app)
        vcmd = (self.entry_pc.register(self.onValidate), '%S')
        self.entry_pc.configure(validate='key', vcmd=vcmd)
        self.entry_pc.grid(row=0, column=1)
        self.entry_pc.insert(0, '1')

        label_pm = tk.Label(frame_app, text='pm')
        label_pm.grid(row=1, column=0)
        self.entry_pm = tk.Entry(frame_app)
        vcmd = (self.entry_pm.register(self.onValidate), '%S')
        self.entry_pm.configure(validate='key', vcmd=vcmd)
        self.entry_pm.grid(row=1, column=1)
        self.entry_pm.insert(0, '0.278')

        label_lmbd = tk.Label(frame_app, text='lambda')
        label_lmbd.grid(row=2, column=0)
        self.entry_lmbd = tk.Entry(frame_app)
        vcmd = (self.entry_lmbd.register(self.onValidate), '%S')
        self.entry_lmbd.configure(validate='key', vcmd=vcmd)
        self.entry_lmbd.grid(row=2, column=1)
        self.entry_lmbd.insert(0, '50')

        label_mu = tk.Label(frame_app, text='mu')
        label_mu.grid(row=3, column=0)
        self.entry_mu = tk.Entry(frame_app)
        vcmd = (self.entry_mu.register(self.onValidate), '%S')
        self.entry_mu.configure(validate='key', vcmd=vcmd)
        self.entry_mu.grid(row=3, column=1)
        self.entry_mu.insert(0, '50')

        label_alpha = tk.Label(frame_app, text='alpha')
        label_alpha.grid(row=4, column=0)
        self.entry_alpha = tk.Entry(frame_app)
        vcmd = (self.entry_alpha.register(self.onValidate), '%S')
        self.entry_alpha.configure(validate='key', vcmd=vcmd)
        self.entry_alpha.grid(row=4, column=1)
        self.entry_alpha.insert(0, '0.33')

        label_beta = tk.Label(frame_app, text='beta')
        label_beta.grid(row=5, column=0)
        self.entry_beta = tk.Entry(frame_app)
        vcmd = (self.entry_beta.register(self.onValidate), '%S')
        self.entry_beta.configure(validate='key', vcmd=vcmd)
        self.entry_beta.grid(row=5, column=1)
        self.entry_beta.insert(0, '0.33')

        label_c_max = tk.Label(frame_app, text='c_max')
        label_c_max.grid(row=6, column=0)
        self.entry_c_max = tk.Entry(frame_app)
        vcmd = (self.entry_c_max.register(self.onValidate), '%S')
        self.entry_c_max.configure(validate='key', vcmd=vcmd)
        self.entry_c_max.grid(row=6, column=1)
        self.entry_c_max.insert(0, '3')

        label_nrep = tk.Label(frame_app, text='nrep')
        label_nrep.grid(row=7, column=0)
        self.entry_nrep = tk.Entry(frame_app)
        vcmd = (self.entry_nrep.register(self.onValidate), '%S')
        self.entry_nrep.configure(validate='key', vcmd=vcmd)
        self.entry_nrep.grid(row=7, column=1)
        self.entry_nrep.insert(0, '100')

        button_execute = ttk.Button(frame_app, text="実行", command=self.mdl_exec)
        button_execute.grid(row=8, column=0)
        
        self.app_mdl_param.mainloop()

    def mdl_exec(self):
        pc = self.entry_pc.get()
        pm = self.entry_pm.get()
        lmbd = self.entry_lmbd.get()
        mu = self.entry_mu.get()
        alpha = self.entry_alpha.get()
        beta = self.entry_beta.get()
        c_max = self.entry_c_max.get()
        nrep = self.entry_nrep.get()

        # DSMの要素値を最大値で正規化する
        DSM_values_tmp = np.copy(self.app_DSM.DSM_values)
        value_max = DSM_values_tmp.max()
        DSM_values_tmp = DSM_values_tmp / value_max

        mdl = MinimumDescriptionLength(
                self.app_DSM.DSM_elms, 
                DSM_values_tmp,
                pc, pm, lmbd, mu, alpha, beta, c_max, nrep)
        mdl.main()
        # 再配列されたDSMを表示する
        # self.app_DSM.import_DSM(tcc.dsm_ordered.astype('int64'), tcc.label_ordered)

        self.app_mdl_param.destroy()
        messagebox.showinfo('', 'クラスタリング(MDL)が完了し，実行ログを出力しました')

    def evaluate_tcc_click(self):
        messagebox.showinfo('', '評価')
        self.app_tcc_param = tk.Tk()       
        self.app_tcc_param.title("評価(Total Coordination)パラメータ")
        self.app_tcc_param.geometry("500x300")
        self.app_tcc_param.attributes("-topmost", True)

        # メインフレームの作成と設置
        vcmd = self.register(self.onValidate)
        
        frame_app = ttk.Frame(self.app_tcc_param)
        frame_app.pack()
        label_npow_cc = tk.Label(frame_app, text='npow_cc')
        label_npow_cc.grid(row=0, column=0)
        self.entry_npow_cc = tk.Entry(frame_app)
        vcmd = (self.entry_npow_cc.register(self.onValidate), '%S')
        self.entry_npow_cc.configure(validate='key', vcmd=vcmd)
        self.entry_npow_cc.grid(row=0, column=1)
        self.entry_npow_cc.insert(0, 1)

        button_execute = ttk.Button(frame_app, text="クラスターマトリクス指定", command=self.evaluate_tcc_file_path)
        button_execute.grid(row=1, column=0)
        label_file_name = tk.Label(frame_app, text='ファイル名')
        label_file_name.grid(row=2, column=0)
        var=tk.StringVar()
        label_file_name_value = tk.Label(frame_app, text=var)
        label_file_name_value.grid(row=2, column=1)

        button_execute = ttk.Button(frame_app, text="実行", command=self.evaluate_tcc)
        button_execute.grid(row=3, column=0)

        return
    
    def evaluate_tcc_file_path(self):
        self.filepath = filedialog.askopenfilename(
            title = "クラスターマトリクスファイルを開く",
            filetypes=[("csv files","*.csv")],
            initialdir = os.getcwd()
            )
        
        # １行目に要素ラベル，２行目以降に値となっているかチェックする

        df = pd.read_csv(self.filepath, header = 0, index_col=0)
        self.cluster_matrix = df.values

        messagebox.showinfo('', 'クラスターマトリクスが指定されました')
        return
    
    def evaluate_tcc(self):
        npow_cc = self.entry_npow_cc.get()

        tcc = TotalCoordinationCost(
                self.app_DSM.DSM_elms, 
                self.app_DSM.DSM_values,
                npow_cc)
        tcc.cluster_matrix = self.cluster_matrix
        cost = tcc.total_coordination_cost()

        # 再配列されたDSMを表示する
        self.app_DSM.import_DSM(tcc.dsm_ordered.astype('int64'), tcc.label_ordered)

        self.app_tcc_param.destroy()
        messagebox.showinfo('', '評価(TCC)が完了し，実行ログを出力しました')



    def evaluate_mdl_click(self):
        messagebox.showinfo('', '評価')
        


    def pyviz_click(self, event=None):

        df = pd.DataFrame(self.app_DSM.DSM_values, columns=self.app_DSM.DSM_elms, index=self.app_DSM.DSM_elms)
        mask_df = df.mask(np.triu(np.ones(df.shape)).astype(bool), None)
        # エッジリストを生成
        edge_lists = mask_df.stack().reset_index().apply(tuple, axis=1).values
        # networkxのグラフを作成
        G = nx.Graph()
        G.add_weighted_edges_from(edge_lists)
        # networkxグラフをpyvisグラフに変換
        g = Network()
        g.from_nx(G)

        # weightに応じて、エッジの太さを変更
        edges_rem = [] #除外するedgeをリスト化
        for i, edge in enumerate(g.edges):
            if abs(g.edges[i]['width']) > 0.1:
                g.edges[i]['width'] = abs(g.edges[i]['width'])*5
            else:
                edges_rem.append(edge)
        for edge in edges_rem:
            g.edges.remove(edge)

        # ボタンやバーなどのGUIを有効化
        g.show_buttons(filter_=['physics', 'nodes'])  # 一部の機能のみ使用
        # g.show_buttons(True)   # 全機能使用

        # グラフをhtmlで表示
        os.makedirs('./out', exist_ok=True)
        now = datetime.datetime.now().strftime(r'%Y%m%d_%H%M%S') # 現在時刻を年月曜日で表示
        g.show("./out/dsm_pyviz_" + now + '.html')
        messagebox.showinfo('', 'グラフ(pyviz)が出力されました')


    def menu_disp1_click(self):
        print("「表示１」が選択された")
        print(f"チェック状態は{self.disp1_value.get()}")

    def menu_disp2_click(self):
        print("「表示２」が選択された")
        print(f"チェック状態は{self.disp2_value.get()}")

    def menu_disp3_click(self):
        print("「表示３」が選択された")
        print(f"チェック状態は{self.disp3_value.get()}")
    
    def menu_select_click(self):
        print(self.radio_val.get(), "番目のラジオボタンが選択されました。")


In [11]:
def sort_DSM(values, elms, idx, direction):
    
    values_ = values.copy()
    elms_ = elms.copy()

    # 単位行列の作成
    p = np.identity(len(elms_), dtype='int64')

    idx_change = idx + direction

    # DSMの並び替え
    p[idx][idx]=0
    p[idx][idx_change]=1
    p[idx_change][idx_change]=0
    p[idx_change][idx]=1
    values_ = np.dot(np.dot(p, values_), p.T)

    # 要素名の並び替え
    # https://qiita.com/koshian2/items/c5f2fb548b9bffe80fdf
    elms_[idx], elms_[idx_change] = elms_[idx_change], elms_[idx]

    return (values_, elms_)

# メイン処理部

In [12]:
if __name__ == '__main__':
    root = tk.Tk()              #ウィンドウを定義
    menu = AppMenu(root)
    root.mainloop()

In [13]:
now = datetime.datetime.now().strftime(r'%Y%m%d_%H%M%S') # 現在時刻を年月曜日で表示
print(now)


20230301_231209
